In [1]:
if True:
    from unsloth import FastVisionModel
    model, tokenizer = FastVisionModel.from_pretrained(
        model_name = "VAT_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = True, # Set to False for 16bit LoRA
    )
    FastVisionModel.for_inference(model) # Enable for inference!

import re, json, ast
from typing import Tuple, Any, List, Optional

def repair_json(s: str, schema: Optional[dict] = None) -> Tuple[str, Any, List[str]]:
    """
    將「幾乎 JSON」的字串修復成合法 JSON。
    回傳: (fixed_text, obj, logs)
      fixed_text: 修復後的 JSON 字串
      obj:        對應的 Python 物件 (dict/list)
      logs:       修復步驟紀錄
    """
    logs: List[str] = []
    text = s.strip()

    # 0) 去掉 ```json ... ``` 或一般 ``` 區塊外殼
    if "```" in text:
        text = re.sub(r"```(?:json|JSON)?", "", text)
        text = text.replace("```", "")
        text = text.strip()
        logs.append("removed code fences")

    # 1) 只擷取最外層 {...} 或 [...]
    def _extract_json_region(t: str) -> str:
        lb, rb = t.find("{"), t.rfind("}")
        if lb != -1 and rb != -1 and rb > lb:
            return t[lb:rb+1]
        lb, rb = t.find("["), t.rfind("]")
        if lb != -1 and rb != -1 and rb > lb:
            return t[lb:rb+1]
        return t

    text2 = _extract_json_region(text)
    if text2 != text:
        logs.append("extracted outer JSON-like region")
        text = text2

    # 2) 嘗試標準 JSON
    try:
        obj = json.loads(text)
        logs.append("parsed by json")
        if schema:
            from jsonschema import validate
            validate(obj, schema); logs.append("validated by jsonschema")
        return json.dumps(obj, ensure_ascii=False, indent=2), obj, logs
    except Exception as e:
        logs.append(f"json.loads failed: {e}")

    # 3) 用 ast.literal_eval 吃單引號/尾逗號 等 Python 字面量
    try:
        obj = ast.literal_eval(text)
        logs.append("parsed by ast.literal_eval")
        if schema:
            from jsonschema import validate
            validate(obj, schema); logs.append("validated by jsonschema")
        return json.dumps(obj, ensure_ascii=False, indent=2), obj, logs
    except Exception as e:
        logs.append(f"ast.literal_eval failed: {e}")

    # 4) 常見修補：彎引號→直引號、True/False/None→JSON、刪尾逗號、單引號→雙引號
    fixed = text.translate(str.maketrans({
        "\u201c": '"', "\u201d": '"', "\u2018": "'", "\u2019": "'",
    }))
    if fixed != text:
        logs.append("normalized curly quotes")

    # Python 常量 → JSON 常量
    fixed2 = re.sub(r'(?<!")\bTrue\b(?!")', "true", fixed)
    fixed2 = re.sub(r'(?<!")\bFalse\b(?!")', "false", fixed2)
    fixed2 = re.sub(r'(?<!")\bNone\b(?!")', "null", fixed2)
    if fixed2 != fixed:
        logs.append("converted Python literals to JSON")
    fixed = fixed2

    # 移除尾逗號
    no_trailing_commas = re.sub(r",\s*([}\]])", r"\1", fixed)
    if no_trailing_commas != fixed:
        logs.append("removed trailing commas")
    fixed = no_trailing_commas

    # 粗略：單引號 → 雙引號（在中文內容通常安全）
    dq = re.sub(r"(?<!\\)'", '"', fixed)
    if dq != fixed:
        logs.append("replaced single quotes with double quotes")
    fixed = dq

    # 5) 最終嘗試標準 JSON
    try:
        obj = json.loads(fixed)
        logs.append("fixed manually then parsed by json")
        if schema:
            from jsonschema import validate
            validate(obj, schema); logs.append("validated by jsonschema")
        return json.dumps(obj, ensure_ascii=False, indent=2), obj, logs
    except Exception as e:
        logs.append(f"final json.loads failed: {e}")
        # 兜底：包 raw
        fallback = {"raw": s}
        if schema:
            logs.append("returned raw because schema validation/parse failed")
        return json.dumps(fallback, ensure_ascii=False, indent=2), fallback, logs

c:\Users\user\anaconda3\envs\VAT\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\user\anaconda3\envs\VAT\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\user\anaconda3\envs\VAT\Lib\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\user\anaconda3\envs\VAT\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\user\anaconda3\envs\VAT\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa4 in position 7: invalid start byte


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W0916 17:17:42.837000 672 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


c:\Users\user\anaconda3\envs\VAT\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:339: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.9.4: Fast Qwen2_5_Vl patching. Transformers: 4.56.1.
   \\   /|    NVIDIA GeForce RTX 4070. Num GPUs = 1. Max memory: 11.994 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
from pathlib import Path
import json
from PIL import Image

FastVisionModel.for_inference(model)  # inference 模式


def chat_once(image):
    instruction = "你是發票/單據分類器與結構化抽取器，請辨識這張文件"

    messages = [
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": instruction}
        ]}
    ]

    # 準備輸入
    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt=True)
    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens=False,
        return_tensors="pt",
    ).to("cuda")

    # 產生（不使用 streamer，改成一次取回）
    gen_ids = model.generate(
        **inputs,
        max_new_tokens=1024,
        use_cache=True,
        temperature=0.1,
        min_p=0.1,
        do_sample=True,              # 若你想要可重現，可改成 False
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

    # 只取「模型新產生」的 token，排除提示部分
    prompt_len = inputs["input_ids"].shape[1]
    new_token_ids = gen_ids[0, prompt_len:]

    output_text = tokenizer.decode(new_token_ids, skip_special_tokens=True).strip()

    try:
        print("json is good!")
        result = json.loads(output_text)
    except Exception:
        print("bad json is repaired!")
        result, obj, logs = repair_json(output_text)

    print(result)

image = Image.open("./invoice2.jpg").convert("RGB")
chat_once(image)


json is good!
bad json is repaired!
{
  "gt_parse": {
    "body": {
      "Abstract": "手提袋 172 55.22 9498",
      "BuyerName": "53812386",
      "BuyerTaxIDNumber": null,
      "CompanyAddress": "台南市永康區崑山街183巷23號",
      "CompanyName": "柏格文具禮品股份有限公司",
      "CompanyTaxIDNumber": "28652798",
      "InvoiceDay": "15",
      "InvoiceMonth": "12",
      "InvoiceYear": "110",
      "PhoneNumber": null
    },
    "doc_class": "triple_receipt",
    "header": {
      "BuyerName": null,
      "BuyerTaxIDNumber": null,
      "InvoiceDay": null,
      "InvoiceMonth": null,
      "InvoiceNumber": "23747161",
      "InvoiceYear": null,
      "PrefixTwoLetters": "TC"
    },
    "rationale": "收銀機統一發票(三聯副聯式扣抵聯)",
    "tail": {
      "CompanyAddress": null,
      "CompanyName": null,
      "CompanyTaxIDNumber": null,
      "InvoiceDay": null,
      "InvoiceMonth": null,
      "InvoiceNumber": null,
      "InvoicePrefixOneLetter": null,
      "InvoiceYear": null,
      "SalesTax": "475",
      "SalesTot

In [ ]:
import os
import json
from tqdm import tqdm

root_dir = "C:/Users/user/pythonproject/AllDataset/VAT-OCR/triple_invoice/"
mode = 'train' #train, test
label_dir = os.path.join(root_dir, 'label', mode)
image_dir = os.path.join(root_dir, 'image')
entries = os.listdir(label_dir)
print(f"Total entries found: {len(entries)}")

for entry in tqdm(entries, desc="Processing entries"):
    print(entry)
    json_path = os.path.join(label_dir, entry)
    img_name = os.path.splitext(entry)[0] + '.jpg'
    img_path = os.path.join(image_dir, img_name)
    #print(img_path)
    
    image = Image.open(img_path).convert("RGB")
    chat_once(image)

Total entries found: 1258


Processing entries:   0%|          | 0/1258 [00:00<?, ?it/s]

1.json


Processing entries:   0%|          | 1/1258 [00:18<6:22:34, 18.26s/it]

json is good!
bad json is repaired!
{
  "gt_parse": {
    "body": {
      "Abstract": "零件2批 25780",
      "BuyerName": null,
      "BuyerTaxIDNumber": null,
      "CompanyAddress": null,
      "CompanyName": null,
      "CompanyTaxIDNumber": null,
      "InvoiceDay": null,
      "InvoiceMonth": null,
      "InvoiceYear": null
    },
    "doc_class": "triple_receipt",
    "header": {
      "BuyerName": "建印貿易有限公司",
      "BuyerTaxIDNumber": "12361988",
      "InvoiceDay": "6",
      "InvoiceMonth": "3",
      "InvoiceNumber": "54957806",
      "InvoiceYear": "112",
      "PrefixTwoLetters": "KY"
    },
    "rationale": "統一發票(三聯式)",
    "tail": {
      "CompanyAddress": "中山區新生北路3段93巷18號",
      "CompanyName": "暉汽材有限公司",
      "CompanyTaxIDNumber": "12868673",
      "SalesTax": "1289",
      "SalesTotalAmount": "25780",
      "TotalAmount": "27069"
    }
  }
}
10.json


...

json is good!
bad json is repaired!
{
  "gt_parse": {
    "body": {
      "Abstract": "177900",
      "BuyerName": null,
      "BuyerTaxIDNumber": null,
      "CompanyAddress": null,
      "CompanyName": null,
      "CompanyTaxIDNumber": null,
      "InvoiceDay": null,
      "InvoiceMonth": null,
      "InvoiceYear": null,
      "PhoneNumber": null,
      "SalesTax": null
    },
    "doc_class": "triple_receipt",
    "header": {
      "BuyerName": "威建企業(股)公司",
      "BuyerTaxIDNumber": "86869795",
      "InvoiceDay": "20",
      "InvoiceMonth": "1",
      "InvoiceNumber": "52064150",
      "InvoiceYear": "113",
      "PrefixTwoLetters": "VY"
    },
    "rationale": "統一發票(三聯式)",
    "tail": {
      "CompanyAddress": "北市松山區德路3段212號8樓",
      "CompanyName": "春廣鋼材有限公司",
      "CompanyTaxIDNumber": "53556914",
      "PhoneNumber": "0926-232167",
      "SalesTax": "8895",
      "SalesTotalAmount": "177900",
      "TotalAmount": "186795"
    }
  }
}
58.json


Processing entries:   5%|▌         | 65/1258 [20:17<5:42:10, 17.21s/it]

json is good!
bad json is repaired!
{
  "gt_parse": {
    "body": {
      "Abstract": "流材 1 3200",
      "BuyerName": null,
      "BuyerTaxIDNumber": null,
      "CompanyAddress": null,
      "CompanyName": null,
      "CompanyTaxIDNumber": null,
      "InvoiceDay": null,
      "InvoiceMonth": null,
      "InvoiceYear": null
    },
    "doc_class": "triple_receipt",
    "header": {
      "BuyerName": "建邦貿易有限公司",
      "BuyerTaxIDNumber": "12361788",
      "InvoiceDay": "10",
      "InvoiceMonth": "4",
      "InvoiceNumber": "14392809",
      "InvoiceYear": "112",
      "PrefixTwoLetters": "KY"
    },
    "rationale": "統一發票(三聯式)"
  }
}
583.json


Processing entries:   5%|▌         | 66/1258 [20:34<5:41:07, 17.17s/it]

json is good!
bad json is repaired!
{
  "gt_parse": {
    "body": {
      "Abstract": "清潔費 1500 18000",
      "BuyerName": null,
      "BuyerTaxIDNumber": null,
      "CompanyAddress": null,
      "CompanyName": null,
      "CompanyTaxIDNumber": null,
      "InvoiceDay": null,
      "InvoiceMonth": null,
      "InvoiceYear": null
    },
    "doc_class": "triple_receipt",
    "header": {
      "BuyerName": "威建企業股份有限公司",
      "BuyerTaxIDNumber": "86869795",
      "InvoiceDay": "25",
      "InvoiceMonth": "1",
      "InvoiceYear": "113",
      "PrefixTwoLetters": "VY"
    },
    "rationale": "統一發票(三聯式)",
    "tail": {
      "CompanyAddress": "桃園市八德區茄明里明光街49巷3號",
      "CompanyName": "固旺有限公司",
      "CompanyTaxIDNumber": "12957106",
      "SalesTax": "900",
      "SalesTotalAmount": "18000",
      "TotalAmount": "18900"
    }
  }
}
584.json


(old inference)

In [ ]:
from PIL import Image

image = Image.open("./invoice.jpg").convert("RGB")
instruction = "你是發票/單據分類器與結構化抽取器，請辨識這張文件"

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

{'gt_parse': {'Other': ['柏格文具禮品股份有限公司', '台南市永康區崙山街183巷23號', '統編', '710'], 'Tail': {'SalesTax': '475', 'SalesTotalAmount': '9498', 'TotalAmount': '9973'}, 'body': {'BuyerTaxIDNumber': '53812386', 'CompanyTaxIDNumber': '28652798'}, 'header': {'InvoiceDay': '15', 'InvoiceMonth': '12', 'InvoiceYear': '110', 'PrefixTwoLetters': 'TC'}}}<|im_end|>
